In [ ]:
%pip install langchain
%pip install langchain-groq
%pip install googlesearch-python

In [ ]:
GROQ_API_KEY = ''  # Put your API_Key generate using groqcloud site inside the single quotes

# **Initialize and Invoke the LLM**

The ChatGroq object is initialized, allowing you to invoke or use the LLMs.
Parameters:

1.  Model: Specifies the LLM model to be used.
2.  Temperature: Determines how variable the LLM's responses should be. A value closer to 0 makes the LLM provide the most probable (least creative) responses consistently.
3.  API_Key: The API key from Groq Cloud.


In [ ]:
# Initialize the LLM Object
from langchain_groq import ChatGroq
llm = ChatGroq(model='llama-3.3-70b-versatile',
               temperature=0, api_key=GROQ_API_KEY)

In [ ]:
# Invoke the LLM with a query
response = llm.invoke('What is an Agent')
response.content

"An agent can refer to different concepts depending on the context. Here are a few common meanings:\n\n1. **Intelligent Agent**: In artificial intelligence, an agent is a program or system that can perceive its environment, make decisions, and take actions to achieve a goal. Agents can be simple or complex, and they can be used in various applications, such as robotics, gaming, or expert systems.\n2. **Software Agent**: A software agent is a computer program that performs tasks on behalf of a user or another program. Examples include web crawlers, chatbots, and virtual assistants like Siri or Alexa.\n3. **Human Agent**: In social sciences, an agent is an individual who acts on behalf of another person or organization. For example, a real estate agent represents a client in a property transaction, while a literary agent represents an author in negotiations with publishers.\n4. **Chemical Agent**: In chemistry, an agent is a substance that causes a chemical reaction or affects the proper

# **Creating and Using an Agent**

Although LLMs are excellent at generating text, they cannot access external tools to perform more complex actions. To address this, we define agents, which consist of an LLM, tools, and an orchestration layer. Agents can autonomously complete complex tasks.

## Components of an Agent (Cognitive Architecture of an Agent)

1. **LLM**: The LLM model used to generate responses, reason, and utilize tools if necessary to complete an action.
2. **Tools**: Functions, extensions, APIs, and other resources that the agent can use to accomplish specific tasks.
3. **Orchestration Layer**: The "brain" that manages the LLM and tools, ensuring that tasks are handled cyclically by the LLM and tools until the objective is achieved.


In [ ]:
# Defining the tools that the LLM can use
from langchain.tools import tool
from googlesearch import search

# Define the add function as a tool


@tool
def add_numbers(query: str) -> str:
    """
    Adds two numbers. Input should be in the format '<num1> <num2>'.
    """
    try:
        print('The add function was invoked')
        num1, num2 = map(int, query.split())
        return str(num1 + num2)
    except ValueError:
        return "Please provide input in the format '<num1> <num2>'."

# Define a Google search function as a tool


@tool
def google_search(query: str) -> str:
    """
    Searches Google for the given query and returns the top 5 results.
    Input: A search query string.
    Output: A string with the top 5 result URLs.
    """
    try:
        print('The Google Search tool was invoked')
        # Perform a Google search and retrieve the top 5 results
        results = search(query, num_results=5)
        # Join the results into a single string
        return "\n".join(results)
    except Exception as e:
        return f"An error occurred during the search: {e}"

In [ ]:
# Binding the tools to the LLM in order to create an agent
from langchain.agents import initialize_agent
agent = initialize_agent(llm=llm, tools=[add_numbers, google_search])

In [ ]:
# Utilizing the agent - Here the agent will invoke Google Search in order to get the right answer
# Other questions - CM of Karnataka, What was the result of Border Gavaskar Trophy Series?


response = agent.invoke('Who is the current PM of India')


response['output']

The Google Search tool was invoked


'Narendra Modi'

In [ ]:
# Using only an llm will not give the right answer
response = llm.invoke(
    'What is the result of the Border Gavaskar Trophy Series')
response.content

'The Border-Gavaskar Trophy is a Test cricket series played between India and Australia. The result of the series can vary depending on the specific series you are referring to, as it is a recurring tournament.\n\nTo provide a more accurate answer, I would need to know the specific series you are asking about, such as the year or the location of the series. However, I can provide some general information about the Border-Gavaskar Trophy.\n\nThe Border-Gavaskar Trophy is named after the legendary cricketers Allan Border of Australia and Sunil Gavaskar of India. The trophy is awarded to the winner of the Test series between the two countries. The series is usually played every two years, with the location alternating between India and Australia.\n\nIf you could provide more context or specify the series you are interested in, I would be happy to try and provide the result of that particular series.'